In [8]:
import pandas as pd
df = pd.read_csv("2023_spotify_ds1.csv")


In [9]:
dataset = []
len(df['pid'].unique())
for i in df['pid'].unique():
    temp = []
    for j in df[df['pid'] == i].index:
        temp.append(df['track_name'][j])
    dataset.append(temp)


In [10]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df_t = pd.DataFrame(te_ary, columns=te.columns_)
df_t

,$ave Dat Money (feat. Fetty Wap & Rich Homie Quan),'Till I Collapse,(Don't Fear) The Reaper,(I Can't Get No) Satisfaction - Mono Version / Remastered 2002,(I Just) Died in Your Arms,...Baby One More Time,0 To 100 / The Catch Up,1-800-273-8255,10 Bands,100 (feat. Drake),...,i,"i hate u, i love u (feat. olivia o'brien)",iSpy (feat. Lil Yachty),m.A.A.d city,oui,pick up the phone,rockstar,through the late night,wokeuplikethis*,wonderful
0,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,True,False,False,False,...,False,False,True,False,False,False,True,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,True,False,False,False,False,...,False,True,True,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,True,True,False,...,False,False,True,False,False,True,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2257,False,False,False,False,False,False,False,False,False,True,...,True,False,True,False,False,True,False,False,False,False
2258,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2259,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2260,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [22]:
from mlxtend.frequent_patterns import fpgrowth, apriori, association_rules

freqItemSet = apriori(df_t, min_support=0.06,use_colnames=True)
rules = association_rules(freqItemSet, metric="confidence", min_threshold=0.05)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(1-800-273-8255),(HUMBLE.),0.100354,0.196286,0.060124,0.599119,3.052268,0.040426,2.004868,0.747377
1,(HUMBLE.),(1-800-273-8255),0.196286,0.100354,0.060124,0.306306,3.052268,0.040426,1.296893,0.836585
2,(2 Phones),(Broccoli (feat. Lil Yachty)),0.108311,0.203360,0.069850,0.644898,3.171216,0.047824,2.243412,0.767828
3,(Broccoli (feat. Lil Yachty)),(2 Phones),0.203360,0.108311,0.069850,0.343478,3.171216,0.047824,1.358201,0.859439
4,(2 Phones),(Jumpman),0.108311,0.179929,0.061008,0.563265,3.130482,0.041520,1.877732,0.763226
...,...,...,...,...,...,...,...,...,...,...
2703,"(XO TOUR Llif3, Mask Off)","(goosebumps, HUMBLE.)",0.101238,0.104332,0.060124,0.593886,5.692251,0.049561,2.205461,0.917175
2704,(goosebumps),"(HUMBLE., XO TOUR Llif3, Mask Off)",0.149425,0.084439,0.060124,0.402367,4.765203,0.047507,1.531979,0.928955
2705,(HUMBLE.),"(goosebumps, XO TOUR Llif3, Mask Off)",0.196286,0.070292,0.060124,0.306306,4.357641,0.046326,1.340229,0.958697
2706,(XO TOUR Llif3),"(goosebumps, HUMBLE., Mask Off)",0.151194,0.074713,0.060124,0.397661,5.322537,0.048828,1.536157,0.956778


In [23]:
import os
import pickle
#save the rules as pickle
path = "/home/zz264/project2-pv/rec/rules_ap.pickle"

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(path), exist_ok=True)

# Save the rules as pickle
with open(path, "wb") as file:
    pickle.dump(rules, file)



In [5]:
import pandas as pd
import os
import sys
import requests
import pandas as pd
import numpy as np
from io import StringIO
import pickle
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

df = pd.read_csv("2023_spotify_ds1.csv")
training_dict = {}
for i in df['pid'].unique():
    training_dict[i] = []

for i in range(len(df)):
    training_dict[df['pid'][i]].append(df['track_name'][i])

te = TransactionEncoder()
te_arr = te.fit_transform(list(training_dict.values()))
freqItemSet = apriori(pd.DataFrame(te_arr, columns=te.columns_), min_support=0.05, use_colnames=True)

# Generate association rules
rules = association_rules(freqItemSet, metric="confidence", min_threshold=0.05)


In [6]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(0 To 100 / The Catch Up),(Jumpman),0.092396,0.179929,0.057029,0.617225,3.430375,0.040404,2.142435,0.780612
1,(Jumpman),(0 To 100 / The Catch Up),0.179929,0.092396,0.057029,0.316953,3.430375,0.040404,1.328758,0.863934
2,(1-800-273-8255),(Congratulations),0.100354,0.164898,0.059240,0.590308,3.579833,0.042691,2.038366,0.801045
3,(Congratulations),(1-800-273-8255),0.164898,0.100354,0.059240,0.359249,3.579833,0.042691,1.404051,0.862958
4,(1-800-273-8255),(HUMBLE.),0.100354,0.196286,0.060124,0.599119,3.052268,0.040426,2.004868,0.747377
...,...,...,...,...,...,...,...,...,...,...
10235,(HUMBLE.),"(Bad and Boujee (feat. Lil Uzi Vert), Bounce B...",0.196286,0.057913,0.050840,0.259009,4.472354,0.039472,1.271387,0.966021
10236,(Bad and Boujee (feat. Lil Uzi Vert)),"(Bounce Back, HUMBLE., XO TOUR Llif3, Mask Off)",0.182140,0.059240,0.050840,0.279126,4.711817,0.040050,1.305028,0.963206
10237,(Bounce Back),"(Bad and Boujee (feat. Lil Uzi Vert), HUMBLE.,...",0.168877,0.065871,0.050840,0.301047,4.570259,0.039716,1.336469,0.939926
10238,(XO TOUR Llif3),"(Bad and Boujee (feat. Lil Uzi Vert), Bounce B...",0.151194,0.065871,0.050840,0.336257,5.104792,0.040881,1.407366,0.947337


In [3]:
import os
import pickle
rules = pickle.load(open("/home/zz264/project2-pv/rec/rules.pickle", "rb")) 

In [4]:
recommend = set()
songs = ['Sorry', 'Love Yourself']
for _, row in rules.iterrows():
    for i in songs:
        if i in row['antecedents']:
            for x in row['consequents']:
                recommend.add(x)

recommend -= set(songs)
list(recommend)

['What Do You Mean?', 'Roses']

In [5]:
df_songs

,artist_name,track_name
0,Sam Hunt,Bottle It Up - Acoustic Mixtape
1,Ryan Gosling,"City Of Stars - From ""La La Land"" Soundtrack"
2,Knife Party,Bonfire
3,Avenged Sevenfold,Welcome to the Family
4,Cheat Codes,Follow You
...,...,...
6969,Pink Floyd,Hey You
6970,Lifehouse,Halfway Gone
6971,Eminem,Won't Back Down
6972,Miley Cyrus,4x4


In [12]:
df

,track_uri,album_name,album_uri,artist_name,artist_uri,duration_ms,pid,track_name
0,spotify:track:06iMqWThw4w8fTFyccvOwr,Country Grammar,spotify:album:2HWBPvQoaMFBF6krXyzpOv,Nelly,spotify:artist:2gBjLmx6zQnFGQJCAQpRgw,291781,115,Ride Wit Me
1,spotify:track:47nm8czanMUzIqHsnr5x61,Clancy's Tavern,spotify:album:6fDc100UHbUvS6IuxBrKRn,Toby Keith,spotify:artist:2bA6fzP0lMAQ4kz6CF61w8,223386,115,Red Solo Cup
2,spotify:track:24NwBd5vZ2CK8VOQVnqdxr,Toys In The Attic,spotify:album:36IxIOGEBAXVozDSiVs09B,Aerosmith,spotify:artist:7Ey4PD4MYsKc5I2dolUwbH,274133,115,Sweet Emotion
3,spotify:track:2WKLUgklz0VDX5bKTZ3eYD,Buy Me A Boat,spotify:album:7BYmCd8JAOHpIiHIDJwDW0,Chris Janson,spotify:artist:60gmlHZmT16wiO9GiXKYK9,179077,115,Buy Me A Boat
4,spotify:track:2d48mamATjby7wgPjdrSOQ,Life Changes,spotify:album:4w5Jvreahp3yvLqc4vCr9I,Thomas Rhett,spotify:artist:6x2LnllRG5uGarZMsD4iO8,223626,115,Craving You
...,...,...,...,...,...,...,...,...
241452,spotify:track:0wdKiSBUT7aZkXUIdJWcwC,Islah,spotify:album:5Hs43ta4vAYKRRRR7DKjt9,Kevin Gates,spotify:artist:1gPhS1zisyXr5dHTYZyiMe,240000,169999,2 Phones
241453,spotify:track:6HTJZ0TQJVMSKkUGzAOe2h,Old Thing Back (feat. Ja Rule and Ralph Tresvant),spotify:album:4v55njNe2owBsm6VuEiTLe,Matoma,spotify:artist:4YXycRbyyAE0wozTk7QMEq,321649,169999,Old Thing Back (feat. Ja Rule and Ralph Tresvant)
241454,spotify:track:6qMXubogFdMrvVMJwqATzd,Jordan Belfort,spotify:album:4G2vHAnHSkuzj1VVBJrRbF,Wes Walker,spotify:artist:0HjYETXAvcL6mzaKjAmH2K,207911,169999,Jordan Belfort
241455,spotify:track:2n5gVJ9fzeX2SSWlLQuyS9,No Label II,spotify:album:56PJDByaunMWwCqs5rV3Nc,Migos,spotify:artist:6oMuImdp5ZcFhWP0ESe6mG,216247,169999,Fight Night


In [1]:
from flask import Flask

app = Flask(__name__)

@app.route("/api/recommend")
def hello_world():
    return "<p>Hello, World!</p>"

In [2]:
import pandas as pd
df_songs = pd.read_csv("2023_spotify_songs.csv")
df_songs

,artist_name,track_name
0,Sam Hunt,Bottle It Up - Acoustic Mixtape
1,Ryan Gosling,"City Of Stars - From ""La La Land"" Soundtrack"
2,Knife Party,Bonfire
3,Avenged Sevenfold,Welcome to the Family
4,Cheat Codes,Follow You
...,...,...
6969,Pink Floyd,Hey You
6970,Lifehouse,Halfway Gone
6971,Eminem,Won't Back Down
6972,Miley Cyrus,4x4
